<img src="https://www.mmu.edu.my/fci/wp-content/uploads/2021/01/FCI_wNEW_MMU_LOGO.png" style="height: 80px;" align=left>  

# Learning Objectives

Towards the end of this lesson, you should be able to:
- write Python codes for association rule mining
- convert data into one hot encoding format

---

This tutorial will be separated into 2 sections, Theoretical questions and practical coding. Complete all the tasks as instructed and compile your files into a single .zip folder. Name your folder using your student ID and lab section as follows: *Lab5_ID_LabSection.zip (e.g. Lab5_1001101100_TT1L.zip)* then submit the file by **13/12/2023 (Wednesday), 11.59pm** for your Week 6 tutorial attendance.

---

# Theoretical Section

Answer the following theoretical questions either within this notebook, or using a separate document editor (Microsoft Word, Google Docs, etc.), or in handwritten form (scan as pdf for submission). Please ensure the answers are accessible if a separate PC is used to run the notebook.

## Question 1

1.	A supermarket transaction database is given as follows. Suppose that the minimum support threshold is 43% (i.e. 3 transactions) and the minimum confidence is 90%:

TID|ITEMS
---|-----
100|1, 2, 3, 4
200|2, 4, 5
300|1, 2, 4, 5
400|1
500|1, 2, 3, 4, 5
600|1, 5
700|2, 3

(a)	Find all frequent itemsets using the Apriori algorithm (Refer Lecture slide 19).

(b)	Find all Association Rules that satisfy the minimum confidence and the minimum support thresholds. <br>
Note: List all rules in a table and show corresponding support and confidence values in separate columns, e.g.:

Associaton rules|Support|Confidence
---|---|---
 $A \rightarrow B$ | s(A $\rightarrow$ B) = ? | c(A $\rightarrow$ B) = ?
 $B \rightarrow A$ | s(B $\rightarrow$ A) = ? | c(B $\rightarrow$ A) = ?
... | ... | ...
... | ... | ...


Reference formula:

<center>
$\sup (A ) = \frac{A}{N}$

$\sup (A \cup B) = \frac{A \cup B}{N}$

$\operatorname{conf}(A \rightarrow B)=\frac{\sup (A \cup B)}{\sup (A)}=\frac{\frac{A \cup B}{N}}{\frac{A}{N}}=\frac{A \cup B}{A}$
</center>

# Question 2

The following contingency table summarizes supermarket transaction data, where $hotdogs$ refer to the transactions containing hot dogs, $\overline{hotdogs}$ refer to the transactions which do not contain hot dogs, hamburgers refer to the transactions containing hamburgers, and $\overline{hamburgers}$ refer to the transactions which do not contain hamburgers.

|           | hotdogs | $\overline{hotdogs}$ | $\sum{row}$ |
|-----------|--------|--------|-------|
| hamburger | 2000 |500|2500
|$\overline{hamburger}$ | 1000 |1500|2500
|$\sum{col}$ | 3000|1500|5000

(a)	Suppose that the association rule “hot dogs $\implies$ hamburgers" is mined. Given a minimum support threshold of 25% and a minimum confidence threshold of 50%, is this association rule strong?

(b)	Based on the given data, is the purchase of hot dogs independent of the purchase of hamburgers? If not, what kind of correlation relationship exists between the two? Choose an appropriate interestingness measure to answer this question.

---
# Practical Section
This section covers the python coding needed to perform association rule mining on a given structured dataset.


## Apyori Package


In [1]:
!pip install apyori 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori



---



### For Google Colab Use Only
Skip this section if you are using Jupyter Notebook etc.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [ ]:
drive_path = '/content/drive/MyDrive/Trimester/2310/TDS3301/Tutorials/Tutorial 5/' #set your google drive path

---
### Importing the Dataset

Performing association rule mining programmatically can be a simple task. For this first example, we will be using the `store_data.csv` dataset containing records of store transactions formatted in `.csv`. Each row represents an individual transaction, while the columns show the items bought. Note that each transaction may not have the same amount of items, therefore, shorter transactions will have a long list of `NaN`. The number of columns is determined by the longest transaction / transaction with most items.

Import and observe the dataset:

In [4]:
try:
  drive_path
except NameError:
  drive_path = ''

store_data = pd.read_csv(drive_path + 'store_data.csv', header=None)
store_data.head()

0          1           2                 3             4   \
0         shrimp    almonds     avocado    vegetables mix  green grapes   
1        burgers  meatballs        eggs               NaN           NaN   
2        chutney        NaN         NaN               NaN           NaN   
3         turkey    avocado         NaN               NaN           NaN   
4  mineral water       milk  energy bar  whole wheat rice     green tea   

                 5     6               7             8             9   \
0  whole weat flour  yams  cottage cheese  energy drink  tomato juice   
1               NaN   NaN             NaN           NaN           NaN   
2               NaN   NaN             NaN           NaN           NaN   
3               NaN   NaN             NaN           NaN           NaN   
4               NaN   NaN             NaN           NaN           NaN   

               10         11     12     13             14      15  \
0  low fat yogurt  green tea  honey  salad  mineral water  salmon   
1             NaN        NaN    NaN    NaN            NaN     NaN   
2             NaN        NaN    NaN    NaN            NaN     NaN   
3             NaN        NaN    NaN    NaN            NaN     NaN   
4             NaN        NaN    NaN    NaN            NaN     NaN   

                  16               17       18         19  
0  antioxydant juice  frozen smoothie  spinach  olive oil  
1                NaN              NaN      NaN        NaN  
2                NaN              NaN      NaN        NaN  
3                NaN              NaN      NaN        NaN  
4                NaN              NaN      NaN        NaN

### Data Preprocessing

The Apriori library we are going to use requires our dataset to be in the form of a list of lists, where the whole dataset is a big list and each transaction in the dataset is an inner list within the outer big list. Currently we have data in the form of a pandas dataframe. To convert our pandas dataframe into a list of lists, execute the following script:

In [5]:
records = []
for i in range(0, 7501):
    records.append([str(store_data.values[i,j]) for j in range(0, 20)])
print(records)

[['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil'], ['burgers', 'meatballs', 'eggs', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['chutney', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['turkey', 'avocado', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], ['low fat yogurt', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan

### Applying Apriori Algorithm

You must set the parameters in the apriori algorithm and input the dataset (now in the form of "list of lists", not dataframe). <br>
The following code executes the algorithm:


In [6]:
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_results = list(association_rules)
display(association_results)

[RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)]),
 RelationRecord(items=frozenset({'mushroom cream sauce', 'escalope'}), support=0.005732568990801226, ordered_statistics=[OrderedStatistic(items_base=frozenset({'mushroom cream sauce'}), items_add=frozenset({'escalope'}), confidence=0.3006993006993007, lift=3.790832696715049)]),
 RelationRecord(items=frozenset({'escalope', 'pasta'}), support=0.005865884548726837, ordered_statistics=[OrderedStatistic(items_base=frozenset({'pasta'}), items_add=frozenset({'escalope'}), confidence=0.3728813559322034, lift=4.700811850163794)]),
 RelationRecord(items=frozenset({'herb & pepper', 'ground beef'}), support=0.015997866951073192, ordered_statistics=[OrderedStatistic(items_base=frozenset({'herb & pepper'}), items_add=frozenset({'ground beef'}), 

### Check the output

The output may not be immediately understandable at this point, but we can do an initial investigation:

In [7]:
#get total number of rules generate using set parameters
len(association_results)

48

In [8]:
#Inspect the first rule
print(association_results[0])

RelationRecord(items=frozenset({'chicken', 'light cream'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)])


### Output in better presentation
The output of the apriori function is not formatted using the typical Association rules notation. We can re-format the output with the following code so that it is easier to be interpreted:

In [9]:
cnt =0

for item in association_results:
    cnt += 1
    # first index of the inner list
    # Contains base item and add item
    pair = item[0]
    items = [x for x in pair]
    print("(Rule " + str(cnt) + ") " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(round(item[1],3)))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(round(item[2][0][2],4)))
    print("Lift: " + str(round(item[2][0][3],4)))
    print("=====================================")

(Rule 1) chicken -> light cream
Support: 0.005
Confidence: 0.2906
Lift: 4.844
(Rule 2) mushroom cream sauce -> escalope
Support: 0.006
Confidence: 0.3007
Lift: 3.7908
(Rule 3) escalope -> pasta
Support: 0.006
Confidence: 0.3729
Lift: 4.7008
(Rule 4) herb & pepper -> ground beef
Support: 0.016
Confidence: 0.3235
Lift: 3.292
(Rule 5) tomato sauce -> ground beef
Support: 0.005
Confidence: 0.3774
Lift: 3.8407
(Rule 6) whole wheat pasta -> olive oil
Support: 0.008
Confidence: 0.2715
Lift: 4.1224
(Rule 7) shrimp -> pasta
Support: 0.005
Confidence: 0.322
Lift: 4.5067
(Rule 8) chicken -> light cream
Support: 0.005
Confidence: 0.2906
Lift: 4.844
(Rule 9) frozen vegetables -> chocolate
Support: 0.005
Confidence: 0.2326
Lift: 3.2545
(Rule 10) cooking oil -> spaghetti
Support: 0.005
Confidence: 0.5714
Lift: 3.282
(Rule 11) mushroom cream sauce -> escalope
Support: 0.006
Confidence: 0.3007
Lift: 3.7908
(Rule 12) pasta -> escalope
Support: 0.006
Confidence: 0.3729
Lift: 4.7008
(Rule 13) frozen veget

---
## Mlxtend Package

Machine Learning Extension (Mlxtend) is yet another library providing useful tools for data science tasks including association rule mining. In this part, let's apply ARM on to the typical structured data.

Firstly, install the package:

In [10]:
!pip install mlxtend

We will be using the `bank-data-final.csv` dataset for this task. It is not a transactional dataset, but we can find associations between the attributes and values using the frequent pattern methods provided by the package.

Let's import the package and load the dataset:

In [11]:
import mlxtend.frequent_patterns as fp

bank_data = pd.read_csv(drive_path + 'bank-data-final.csv')
display(bank_data)

age     sex      region       income married  children  car save_act  \
0     35_51  FEMALE  INNER_CITY      0_24386      NO         1   NO       NO   
1     35_51    MALE        TOWN  24387_43758     YES         3  YES       NO   
2    52_max  FEMALE  INNER_CITY      0_24386     YES         0  YES      YES   
3      0_34  FEMALE        TOWN      0_24386     YES         3   NO       NO   
4    52_max  FEMALE       RURAL    43759_max     YES         0   NO      YES   
..      ...     ...         ...          ...     ...       ...  ...      ...   
595  52_max  FEMALE  INNER_CITY    43759_max      NO         2  YES      YES   
596    0_34  FEMALE  INNER_CITY      0_24386     YES         0  YES      YES   
597    0_34  FEMALE        TOWN      0_24386     YES         0  YES      YES   
598    0_34    MALE  INNER_CITY      0_24386     YES         0   NO      YES   
599   35_51    MALE        TOWN  24387_43758      NO         0  YES       NO   

    current_act mortgage  pep  
0            NO       NO  YES  
1           YES      YES   NO  
2           YES       NO   NO  
3           YES       NO   NO  
4            NO       NO   NO  
..          ...      ...  ...  
595         YES      YES   NO  
596         YES       NO   NO  
597          NO       NO  YES  
598          NO      YES   NO  
599         YES      YES  YES  

[600 rows x 11 columns]

THe dataset contains mostly categorical data, and numerical data like age has already been discretized into bins. However, the mlxtend package requires "one hot encoding" format for all columns in order to be used with the apriori and fpgrowth methods.

One hot encoding: converting every value in the column into their individual columns, then using 0 / 1 or TRUE / FALSE to indicate the category. E.g.:

|Original Column|One|hot|encoding|
|---|---|---|---|
|Colour| Color_Red | Color_Green | Color_Blue|
|Blue|0|0|1|
|Red|1|0|0|
|Green|0|1|0|
|Red|1|0|0|

To do this transformation, we can use the `get_dummies` function from pandas:

In [12]:
bank_data_onehot = pd.get_dummies(bank_data, columns=bank_data.columns)
display(bank_data_onehot)

age_0_34  age_35_51  age_52_max  sex_FEMALE  sex_MALE  region_INNER_CITY  \
0       False       True       False        True     False               True   
1       False       True       False       False      True              False   
2       False      False        True        True     False               True   
3        True      False       False        True     False              False   
4       False      False        True        True     False              False   
..        ...        ...         ...         ...       ...                ...   
595     False      False        True        True     False               True   
596      True      False       False        True     False               True   
597      True      False       False        True     False              False   
598      True      False       False       False      True               True   
599     False       True       False       False      True              False   

     region_RURAL  region_SUBURBAN  region_TOWN  income_0_24386  ...  car_NO  \
0           False            False        False            True  ...    True   
1           False            False         True           False  ...   False   
2           False            False        False            True  ...   False   
3           False            False         True            True  ...    True   
4            True            False        False           False  ...    True   
..            ...              ...          ...             ...  ...     ...   
595         False            False        False           False  ...   False   
596         False            False        False            True  ...   False   
597         False            False         True            True  ...   False   
598         False            False        False            True  ...    True   
599         False            False         True           False  ...   False   

     car_YES  save_act_NO  save_act_YES  current_act_NO  current_act_YES  \
0      False         True         False            True            False   
1       True         True         False           False             True   
2       True        False          True           False             True   
3      False         True         False           False             True   
4      False        False          True            True            False   
..       ...          ...           ...             ...              ...   
595     True        False          True           False             True   
596     True        False          True           False             True   
597     True        False          True            True            False   
598    False        False          True            True            False   
599     True         True         False           False             True   

     mortgage_NO  mortgage_YES  pep_NO  pep_YES  
0           True         False   False     True  
1          False          True    True    False  
2           True         False    True    False  
3           True         False    True    False  
4           True         False    True    False  
..           ...           ...     ...      ...  
595        False          True    True    False  
596         True         False    True    False  
597         True         False   False     True  
598        False          True    True    False  
599        False          True   False     True  

[600 rows x 28 columns]

Notice the dataset now has 28 columns, instead of the initial 11 columns.
Next, we can look for frequent patterns using either the `apriori` method, or the `fpgrowth` method in the following codes.


In [13]:
#Frequent pattern using aprioro method
fp_apriori = fp.apriori(bank_data_onehot,min_support=0.3,use_colnames=True)
display(fp_apriori)

support                                      itemsets
0   0.325000                                    (age_0_34)
1   0.356667                                   (age_35_51)
2   0.318333                                  (age_52_max)
3   0.500000                                  (sex_FEMALE)
4   0.500000                                    (sex_MALE)
5   0.448333                           (region_INNER_CITY)
6   0.475000                              (income_0_24386)
7   0.391667                          (income_24387_43758)
8   0.340000                                  (married_NO)
9   0.660000                                 (married_YES)
10  0.438333                                  (children_0)
11  0.506667                                      (car_NO)
12  0.493333                                     (car_YES)
13  0.310000                                 (save_act_NO)
14  0.690000                                (save_act_YES)
15  0.758333                             (current_act_YES)
16  0.651667                                 (mortgage_NO)
17  0.348333                                (mortgage_YES)
18  0.543333                                      (pep_NO)
19  0.456667                                     (pep_YES)
20  0.325000                     (sex_FEMALE, married_YES)
21  0.343333                    (sex_FEMALE, save_act_YES)
22  0.383333                 (sex_FEMALE, current_act_YES)
23  0.341667                     (mortgage_NO, sex_FEMALE)
24  0.335000                       (sex_MALE, married_YES)
25  0.346667                      (sex_MALE, save_act_YES)
26  0.375000                   (sex_MALE, current_act_YES)
27  0.310000                       (mortgage_NO, sex_MALE)
28  0.341667          (current_act_YES, region_INNER_CITY)
29  0.325000                 (income_0_24386, married_YES)
30  0.358333             (current_act_YES, income_0_24386)
31  0.313333                 (mortgage_NO, income_0_24386)
32  0.300000                     (children_0, married_YES)
33  0.336667                         (car_NO, married_YES)
34  0.323333                        (car_YES, married_YES)
35  0.461667                   (save_act_YES, married_YES)
36  0.488333                (current_act_YES, married_YES)
37  0.435000                    (mortgage_NO, married_YES)
38  0.403333                         (pep_NO, married_YES)
39  0.331667                 (current_act_YES, children_0)
40  0.341667                        (car_NO, save_act_YES)
41  0.391667                     (current_act_YES, car_NO)
42  0.328333                         (mortgage_NO, car_NO)
43  0.348333                       (car_YES, save_act_YES)
44  0.366667                    (current_act_YES, car_YES)
45  0.323333                        (mortgage_NO, car_YES)
46  0.531667               (current_act_YES, save_act_YES)
47  0.450000                   (mortgage_NO, save_act_YES)
48  0.391667                        (save_act_YES, pep_NO)
49  0.501667                (mortgage_NO, current_act_YES)
50  0.406667                     (current_act_YES, pep_NO)
51  0.351667                    (current_act_YES, pep_YES)
52  0.348333                         (mortgage_NO, pep_NO)
53  0.303333                        (mortgage_NO, pep_YES)
54  0.343333  (current_act_YES, save_act_YES, married_YES)
55  0.306667      (mortgage_NO, save_act_YES, married_YES)
56  0.331667   (mortgage_NO, current_act_YES, married_YES)
57  0.353333  (mortgage_NO, current_act_YES, save_act_YES)

In [14]:
#frequent pattern using fpgrowth method
fp_fpgrowth = fp.fpgrowth(bank_data_onehot, min_support=0.3, use_colnames=True)
display(fp_fpgrowth)

support                                      itemsets
0   0.651667                                 (mortgage_NO)
1   0.506667                                      (car_NO)
2   0.500000                                  (sex_FEMALE)
3   0.475000                              (income_0_24386)
4   0.456667                                     (pep_YES)
5   0.448333                           (region_INNER_CITY)
6   0.356667                                   (age_35_51)
7   0.340000                                  (married_NO)
8   0.310000                                 (save_act_NO)
9   0.758333                             (current_act_YES)
10  0.660000                                 (married_YES)
11  0.543333                                      (pep_NO)
12  0.500000                                    (sex_MALE)
13  0.493333                                     (car_YES)
14  0.391667                          (income_24387_43758)
15  0.348333                                (mortgage_YES)
16  0.690000                                (save_act_YES)
17  0.438333                                  (children_0)
18  0.318333                                  (age_52_max)
19  0.325000                                    (age_0_34)
20  0.501667                (mortgage_NO, current_act_YES)
21  0.450000                   (mortgage_NO, save_act_YES)
22  0.435000                    (mortgage_NO, married_YES)
23  0.353333  (mortgage_NO, current_act_YES, save_act_YES)
24  0.331667   (mortgage_NO, current_act_YES, married_YES)
25  0.306667      (mortgage_NO, save_act_YES, married_YES)
26  0.328333                         (mortgage_NO, car_NO)
27  0.391667                     (current_act_YES, car_NO)
28  0.336667                         (car_NO, married_YES)
29  0.341667                        (car_NO, save_act_YES)
30  0.341667                     (mortgage_NO, sex_FEMALE)
31  0.383333                 (sex_FEMALE, current_act_YES)
32  0.343333                    (sex_FEMALE, save_act_YES)
33  0.325000                     (sex_FEMALE, married_YES)
34  0.313333                 (mortgage_NO, income_0_24386)
35  0.358333             (current_act_YES, income_0_24386)
36  0.325000                 (income_0_24386, married_YES)
37  0.303333                        (mortgage_NO, pep_YES)
38  0.351667                    (current_act_YES, pep_YES)
39  0.341667          (current_act_YES, region_INNER_CITY)
40  0.488333                (current_act_YES, married_YES)
41  0.461667                   (save_act_YES, married_YES)
42  0.343333  (current_act_YES, save_act_YES, married_YES)
43  0.406667                     (current_act_YES, pep_NO)
44  0.403333                         (pep_NO, married_YES)
45  0.391667                        (save_act_YES, pep_NO)
46  0.348333                         (mortgage_NO, pep_NO)
47  0.375000                   (sex_MALE, current_act_YES)
48  0.335000                       (sex_MALE, married_YES)
49  0.310000                       (mortgage_NO, sex_MALE)
50  0.346667                      (sex_MALE, save_act_YES)
51  0.366667                    (current_act_YES, car_YES)
52  0.323333                        (car_YES, married_YES)
53  0.348333                       (car_YES, save_act_YES)
54  0.323333                        (mortgage_NO, car_YES)
55  0.531667               (current_act_YES, save_act_YES)
56  0.331667                 (current_act_YES, children_0)
57  0.300000                     (children_0, married_YES)

**Question**: Add time tracking to explore the different processing times. Which is faster?



With the found frequent patterns, we can then generate association rules from those patterns using `association_rules()` function. This function provides different interestingness metrics to evauate the rules, such as `confidence`, `lift`, `conviction`, and `leverage`. Experiment with the different metrics and rules generated from the mining:

In [15]:
assoc_rules = fp.association_rules(fp_apriori,metric='confidence', min_threshold=0.5,support_only=False)
display(assoc_rules)

antecedents                      consequents  \
0                      (sex_FEMALE)                    (married_YES)   
1                      (sex_FEMALE)                   (save_act_YES)   
2                      (sex_FEMALE)                (current_act_YES)   
3                 (current_act_YES)                     (sex_FEMALE)   
4                     (mortgage_NO)                     (sex_FEMALE)   
..                              ...                              ...   
64   (mortgage_NO, current_act_YES)                   (save_act_YES)   
65      (mortgage_NO, save_act_YES)                (current_act_YES)   
66  (current_act_YES, save_act_YES)                    (mortgage_NO)   
67                    (mortgage_NO)  (current_act_YES, save_act_YES)   
68                   (save_act_YES)   (mortgage_NO, current_act_YES)   

    antecedent support  consequent support   support  confidence      lift  \
0             0.500000            0.660000  0.325000    0.650000  0.984848   
1             0.500000            0.690000  0.343333    0.686667  0.995169   
2             0.500000            0.758333  0.383333    0.766667  1.010989   
3             0.758333            0.500000  0.383333    0.505495  1.010989   
4             0.651667            0.500000  0.341667    0.524297  1.048593   
..                 ...                 ...       ...         ...       ...   
64            0.501667            0.690000  0.353333    0.704319  1.020752   
65            0.450000            0.758333  0.353333    0.785185  1.035409   
66            0.531667            0.651667  0.353333    0.664577  1.019811   
67            0.651667            0.531667  0.353333    0.542199  1.019811   
68            0.690000            0.501667  0.353333    0.512077  1.020752   

    leverage  conviction  zhangs_metric  
0  -0.005000    0.971429      -0.029851  
1  -0.001667    0.989362      -0.009615  
2   0.004167    1.035714       0.021739  
3   0.004167    1.011111       0.044978  
4   0.015833    1.051075       0.133038  
..       ...         ...            ...  
64  0.007183    1.048427       0.040796  
65  0.012083    1.125000       0.062178  
66  0.006864    1.038489       0.041479  
67  0.006864    1.023007       0.055769  
68  0.007183    1.021337       0.065581  

[69 rows x 10 columns]

## Exercise

Now that you have tested out two different packages and performed association on different types of dataset, try to apply this onto the BigMartSales dataset to find interesting pattern and association rules. <br>
For this simple exercise, simplify your task as follows:
* Select only the categorical columns from the `train.csv` dataset for this task
* Exlude the `Item_Identifier` and `Outlet_Identifier` columns from this task

In [16]:
#Enter your code here
df = pd.read_csv("BigMartSales/train.csv")
display(df)

Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0              FDA15        9.300          Low Fat         0.016047   
1              DRC01        5.920          Regular         0.019278   
2              FDN15       17.500          Low Fat         0.016760   
3              FDX07       19.200          Regular         0.000000   
4              NCD19        8.930          Low Fat         0.000000   
...              ...          ...              ...              ...   
8518           FDF22        6.865          Low Fat         0.056783   
8519           FDS36        8.380          Regular         0.046982   
8520           NCJ29       10.600          Low Fat         0.035186   
8521           FDN46        7.210          Regular         0.145221   
8522           DRG01       14.800          Low Fat         0.044878   

                  Item_Type  Item_MRP Outlet_Identifier  \
0                     Dairy  249.8092            OUT049   
1               Soft Drinks   48.2692            OUT018   
2                      Meat  141.6180            OUT049   
3     Fruits and Vegetables  182.0950            OUT010   
4                 Household   53.8614            OUT013   
...                     ...       ...               ...   
8518            Snack Foods  214.5218            OUT013   
8519           Baking Goods  108.1570            OUT045   
8520     Health and Hygiene   85.1224            OUT035   
8521            Snack Foods  103.1332            OUT018   
8522            Soft Drinks   75.4670            OUT046   

      Outlet_Establishment_Year Outlet_Size Outlet_Location_Type  \
0                          1999      Medium               Tier 1   
1                          2009      Medium               Tier 3   
2                          1999      Medium               Tier 1   
3                          1998         NaN               Tier 3   
4                          1987        High               Tier 3   
...                         ...         ...                  ...   
8518                       1987        High               Tier 3   
8519                       2002         NaN               Tier 2   
8520                       2004       Small               Tier 2   
8521                       2009      Medium               Tier 3   
8522                       1997       Small               Tier 1   

            Outlet_Type  Item_Outlet_Sales  
0     Supermarket Type1          3735.1380  
1     Supermarket Type2           443.4228  
2     Supermarket Type1          2097.2700  
3         Grocery Store           732.3800  
4     Supermarket Type1           994.7052  
...                 ...                ...  
8518  Supermarket Type1          2778.3834  
8519  Supermarket Type1           549.2850  
8520  Supermarket Type1          1193.1136  
8521  Supermarket Type2          1845.5976  
8522  Supermarket Type1           765.6700  

[8523 rows x 12 columns]

In [33]:
#data preprocessing
cat_col = df.select_dtypes(include=['object']).columns
cat_df = df[cat_col]
cat_df = cat_df.drop('Outlet_Identifier', axis=1)
cat_df = cat_df.drop("Item_Identifier", axis=1)
display(cat_df)

Item_Fat_Content              Item_Type Outlet_Size Outlet_Location_Type  \
0             Low Fat                  Dairy      Medium               Tier 1   
1             Regular            Soft Drinks      Medium               Tier 3   
2             Low Fat                   Meat      Medium               Tier 1   
3             Regular  Fruits and Vegetables         NaN               Tier 3   
4             Low Fat              Household        High               Tier 3   
...               ...                    ...         ...                  ...   
8518          Low Fat            Snack Foods        High               Tier 3   
8519          Regular           Baking Goods         NaN               Tier 2   
8520          Low Fat     Health and Hygiene       Small               Tier 2   
8521          Regular            Snack Foods      Medium               Tier 3   
8522          Low Fat            Soft Drinks       Small               Tier 1   

            Outlet_Type  
0     Supermarket Type1  
1     Supermarket Type2  
2     Supermarket Type1  
3         Grocery Store  
4     Supermarket Type1  
...                 ...  
8518  Supermarket Type1  
8519  Supermarket Type1  
8520  Supermarket Type1  
8521  Supermarket Type2  
8522  Supermarket Type1  

[8523 rows x 5 columns]

In [34]:
cat_onehot = pd.get_dummies(cat_df, columns=cat_df.columns)
display(cat_onehot)

Item_Fat_Content_LF  Item_Fat_Content_Low Fat  Item_Fat_Content_Regular  \
0                   False                      True                     False   
1                   False                     False                      True   
2                   False                      True                     False   
3                   False                     False                      True   
4                   False                      True                     False   
...                   ...                       ...                       ...   
8518                False                      True                     False   
8519                False                     False                      True   
8520                False                      True                     False   
8521                False                     False                      True   
8522                False                      True                     False   

      Item_Fat_Content_low fat  Item_Fat_Content_reg  Item_Type_Baking Goods  \
0                        False                 False                   False   
1                        False                 False                   False   
2                        False                 False                   False   
3                        False                 False                   False   
4                        False                 False                   False   
...                        ...                   ...                     ...   
8518                     False                 False                   False   
8519                     False                 False                    True   
8520                     False                 False                   False   
8521                     False                 False                   False   
8522                     False                 False                   False   

      Item_Type_Breads  Item_Type_Breakfast  Item_Type_Canned  \
0                False                False             False   
1                False                False             False   
2                False                False             False   
3                False                False             False   
4                False                False             False   
...                ...                  ...               ...   
8518             False                False             False   
8519             False                False             False   
8520             False                False             False   
8521             False                False             False   
8522             False                False             False   

      Item_Type_Dairy  ...  Outlet_Size_High  Outlet_Size_Medium  \
0                True  ...             False                True   
1               False  ...             False                True   
2               False  ...             False                True   
3               False  ...             False               False   
4               False  ...              True               False   
...               ...  ...               ...                 ...   
8518            False  ...              True               False   
8519            False  ...             False               False   
8520            False  ...             False               False   
8521            False  ...             False                True   
8522            False  ...             False               False   

      Outlet_Size_Small  Outlet_Location_Type_Tier 1  \
0                 False                         True   
1                 False                        False   
2                 False                         True   
3                 False                        False   
4                 False                        False   
...                 ...                          ...   
8518              False                        False   
8519              False   

In [35]:
#Frequent pattern using aprioro method
fp_apriori = fp.apriori(cat_onehot,min_support=0.3,use_colnames=True)
display(fp_apriori)

support                                           itemsets
0  0.597090                         (Item_Fat_Content_Low Fat)
1  0.338965                         (Item_Fat_Content_Regular)
2  0.327702                               (Outlet_Size_Medium)
3  0.326763                      (Outlet_Location_Type_Tier 2)
4  0.393054                      (Outlet_Location_Type_Tier 3)
5  0.654347                    (Outlet_Type_Supermarket Type1)
6  0.392702  (Item_Fat_Content_Low Fat, Outlet_Type_Superma...
7  0.326763  (Outlet_Location_Type_Tier 2, Outlet_Type_Supe...

In [36]:
#frequent pattern using fpgrowth method
fp_fpgrowth = fp.fpgrowth(cat_onehot, min_support=0.3, use_colnames=True)
display(fp_fpgrowth)

support                                           itemsets
0  0.654347                    (Outlet_Type_Supermarket Type1)
1  0.597090                         (Item_Fat_Content_Low Fat)
2  0.327702                               (Outlet_Size_Medium)
3  0.393054                      (Outlet_Location_Type_Tier 3)
4  0.338965                         (Item_Fat_Content_Regular)
5  0.326763                      (Outlet_Location_Type_Tier 2)
6  0.392702  (Item_Fat_Content_Low Fat, Outlet_Type_Superma...
7  0.326763  (Outlet_Location_Type_Tier 2, Outlet_Type_Supe...

In [37]:
assoc_rules = fp.association_rules(fp_apriori,metric='confidence', min_threshold=0.5,support_only=False)
display(assoc_rules)

antecedents                      consequents  \
0       (Item_Fat_Content_Low Fat)  (Outlet_Type_Supermarket Type1)   
1  (Outlet_Type_Supermarket Type1)       (Item_Fat_Content_Low Fat)   
2    (Outlet_Location_Type_Tier 2)  (Outlet_Type_Supermarket Type1)   

   antecedent support  consequent support   support  confidence      lift  \
0            0.597090            0.654347  0.392702    0.657693  1.005113   
1            0.654347            0.597090  0.392702    0.600143  1.005113   
2            0.326763            0.654347  0.326763    1.000000  1.528241   

   leverage  conviction  zhangs_metric  
0  0.001998    1.009775       0.012627  
1  0.001998    1.007636       0.014718  
2  0.112947         inf       0.513419